In [1]:
import pandas as pd
import numpy as np

In [46]:
#read in the result of query for last 14 days
df_14 = pd.read_csv('/home/jupyter-vladperesad/files/query-14-.csv')
df_14

,student_id,student_name,group_name,date,homework,behaviour,comprehension,vocabulary,speaking,reading,writing
0,1,Ethan,wed1905,2024-02-28,0.0,5,3,3,2,3.0,NaN
1,2,Dora,wed1905,2024-02-28,0.0,5,5,5,4,5.0,NaN
2,3,Eleven,wed1905,2024-02-28,5.0,5,5,4,2,4.0,NaN
3,4,Mason,thu1735,2024-02-29,NaN,5,5,5,4,5.0,5.0
4,5,Elsa,thu1735,2024-02-29,NaN,4,5,2,2,3.0,2.0
5,6,David,thu1735,2024-02-29,NaN,5,5,2,3,5.0,4.0
6,7,Wilson,thu1905,2024-02-22,5.0,4,4,5,4,4.0,NaN
7,7,Wilson,thu1905,2024-02-29,5.0,2,5,5,5,5.0,5.0
8,8,Evan,thu1905,2024-02-22,0.0,3,2,4,4,3.0,NaN
9,8,Evan,thu1905,2024-02-29,0.0,2,4,3,2,5.0,5.0


In [47]:
#create a subset that only contains student_ids
df_st = df_14['student_id']

In [48]:
#calculate percent difference in students performance beween most recent class and the class before that
df_pct = df_14 \
    .drop(columns=['student_name',
                    'group_name',
                    'date']) \
    .groupby('student_id') \
    .pct_change(fill_method=None)*100

In [49]:
#drop rows that have NaN value across all columns (these are the students who came once hence the percentage difference cannot be calculated)
df_pct.dropna(axis=0, how='all', inplace=True)

In [50]:
df_pct

,homework,behaviour,comprehension,vocabulary,speaking,reading,writing
7,0.0,-50.000000,25.000000,0.000000,25.000000,25.000000,NaN
9,NaN,-33.333333,100.000000,-25.000000,-50.000000,66.666667,NaN
11,NaN,33.333333,-60.000000,0.000000,-50.000000,200.000000,NaN
15,NaN,-50.000000,0.000000,100.000000,0.000000,NaN,-80.0
21,-40.0,0.000000,0.000000,0.000000,0.000000,NaN,-25.0
23,NaN,0.000000,0.000000,25.000000,0.000000,NaN,0.0
26,inf,-40.000000,66.666667,66.666667,33.333333,NaN,inf
29,NaN,-20.000000,66.666667,0.000000,0.000000,0.000000,NaN
33,NaN,-20.000000,33.333333,-20.000000,25.000000,0.000000,NaN
35,NaN,-100.000000,66.666667,-100.000000,-33.333333,25.000000,NaN


In [56]:
#add to column that contains stidents ids for later merging
df_pct = df_pct.join(df_st, how='inner')

In [57]:
df_pct

,homework,behaviour,comprehension,vocabulary,speaking,reading,writing,student_id
7,0.0,-50.000000,25.000000,0.000000,25.000000,25.000000,NaN,7
9,NaN,-33.333333,100.000000,-25.000000,-50.000000,66.666667,NaN,8
11,NaN,33.333333,-60.000000,0.000000,-50.000000,200.000000,NaN,9
15,NaN,-50.000000,0.000000,100.000000,0.000000,NaN,-80.0,13
21,-40.0,0.000000,0.000000,0.000000,0.000000,NaN,-25.0,19
23,NaN,0.000000,0.000000,25.000000,0.000000,NaN,0.0,20
26,inf,-40.000000,66.666667,66.666667,33.333333,NaN,inf,22
29,NaN,-20.000000,66.666667,0.000000,0.000000,0.000000,NaN,24
33,NaN,-20.000000,33.333333,-20.000000,25.000000,0.000000,NaN,27
35,NaN,-100.000000,66.666667,-100.000000,-33.333333,25.000000,NaN,28


In [53]:
#read in the result of query for last 7 days
df_7 = pd.read_csv('/home/jupyter-vladperesad/files/query-7-.csv')
df_7

,student_id,student_name,group_name,date,homework,behaviour,comprehension,vocabulary,speaking,reading,writing
0,4,Mason,thu1735,2024-02-29,NaN,5,5,5,4,5.0,5.0
1,5,Elsa,thu1735,2024-02-29,NaN,4,5,2,2,3.0,2.0
2,6,David,thu1735,2024-02-29,NaN,5,5,2,3,5.0,4.0
3,7,Wilson,thu1905,2024-02-29,5.0,2,5,5,5,5.0,5.0
4,8,Evan,thu1905,2024-02-29,0.0,2,4,3,2,5.0,5.0
5,9,Daniel,thu1905,2024-02-29,0.0,4,2,4,2,3.0,3.0
6,11,Cara,fri1905,2024-03-01,3.0,3,3,5,5,NaN,0.0
7,12,Elsa,fri1905,2024-03-01,NaN,4,5,3,5,NaN,NaN
8,13,Charles,fri1905,2024-03-01,0.0,1,5,4,5,NaN,1.0
9,14,Edon,fri1905,2024-03-01,NaN,2,5,1,5,NaN,NaN


In [60]:
final_table = df_7.merge(df_pct, how='left', on='student_id', suffixes=('_act','_pct'))
final_table

,student_id,student_name,group_name,date,homework_act,behaviour_act,comprehension_act,vocabulary_act,speaking_act,reading_act,writing_act,homework_pct,behaviour_pct,comprehension_pct,vocabulary_pct,speaking_pct,reading_pct,writing_pct
0,4,Mason,thu1735,2024-02-29,NaN,5,5,5,4,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,Elsa,thu1735,2024-02-29,NaN,4,5,2,2,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,David,thu1735,2024-02-29,NaN,5,5,2,3,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,Wilson,thu1905,2024-02-29,5.0,2,5,5,5,5.0,5.0,0.0,-50.000000,25.000000,0.000000,25.000000,25.000000,NaN
4,8,Evan,thu1905,2024-02-29,0.0,2,4,3,2,5.0,5.0,NaN,-33.333333,100.000000,-25.000000,-50.000000,66.666667,NaN
5,9,Daniel,thu1905,2024-02-29,0.0,4,2,4,2,3.0,3.0,NaN,33.333333,-60.000000,0.000000,-50.000000,200.000000,NaN
6,11,Cara,fri1905,2024-03-01,3.0,3,3,5,5,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,12,Elsa,fri1905,2024-03-01,NaN,4,5,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,13,Charles,fri1905,2024-03-01,0.0,1,5,4,5,NaN,1.0,NaN,-50.000000,0.000000,100.000000,0.000000,NaN,-80.0
9,14,Edon,fri1905,2024-03-01,NaN,2,5,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
